# Importing Library

In [82]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/playground-series-s4e11/sample_submission.csv
/kaggle/input/playground-series-s4e11/train.csv
/kaggle/input/playground-series-s4e11/test.csv


In [83]:
# !pip install git+https://github.com/AutoViML/Auto_ViML.git
# !pip install ipykernel==5.5.6
# !pip install --upgrade pandas
# !pip install scipy==1.11.0


# Dataset|

In [84]:
df=pd.read_csv('/kaggle/input/playground-series-s4e11/train.csv',index_col=['id'])
df.head()

,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
id,,,,,,,,,,,,,,,,,,,
0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,NaN,5.0,NaN,NaN,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0
1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,NaN,4.0,NaN,NaN,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1
2,Yuvraj,Male,33.0,Visakhapatnam,Student,NaN,5.0,NaN,8.97,2.0,NaN,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,NaN,5.0,NaN,NaN,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1
4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,NaN,1.0,NaN,NaN,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0


In [85]:
df.columns

Index(['Name', 'Gender', 'Age', 'City', 'Working Professional or Student',
       'Profession', 'Academic Pressure', 'Work Pressure', 'CGPA',
       'Study Satisfaction', 'Job Satisfaction', 'Sleep Duration',
       'Dietary Habits', 'Degree', 'Have you ever had suicidal thoughts ?',
       'Work/Study Hours', 'Financial Stress',
       'Family History of Mental Illness', 'Depression'],
      dtype='object')

In [86]:
df.describe()

,Age,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Work/Study Hours,Financial Stress,Depression
count,140700.000000,27897.000000,112782.000000,27898.000000,27897.000000,112790.000000,140700.000000,140696.000000,140700.000000
mean,40.388621,3.142273,2.998998,7.658636,2.944940,2.974404,6.252679,2.988983,0.181713
std,12.384099,1.380457,1.405771,1.464466,1.360197,1.416078,3.853615,1.413633,0.385609
min,18.000000,1.000000,1.000000,5.030000,1.000000,1.000000,0.000000,1.000000,0.000000
25%,29.000000,2.000000,2.000000,6.290000,2.000000,2.000000,3.000000,2.000000,0.000000
50%,42.000000,3.000000,3.000000,7.770000,3.000000,3.000000,6.000000,3.000000,0.000000
75%,51.000000,4.000000,4.000000,8.920000,4.000000,4.000000,10.000000,4.000000,0.000000
max,60.000000,5.000000,5.000000,10.000000,5.000000,5.000000,12.000000,5.000000,1.000000


In [87]:
df.shape

(140700, 19)

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 140700 entries, 0 to 140699
Data columns (total 19 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   Name                                   140700 non-null  object 
 1   Gender                                 140700 non-null  object 
 2   Age                                    140700 non-null  float64
 3   City                                   140700 non-null  object 
 4   Working Professional or Student        140700 non-null  object 
 5   Profession                             104070 non-null  object 
 6   Academic Pressure                      27897 non-null   float64
 7   Work Pressure                          112782 non-null  float64
 8   CGPA                                   27898 non-null   float64
 9   Study Satisfaction                     27897 non-null   float64
 10  Job Satisfaction                       112790 non-null  float

++++++++++++++++++++++++++++++++++++++++++++++++

# Data Processing

In [89]:
# Standardize column names
df.columns = df.columns.str.strip().str.replace(" ", "_").str.lower()

In [90]:
df.isnull().sum()

name                                          0
gender                                        0
age                                           0
city                                          0
working_professional_or_student               0
profession                                36630
academic_pressure                        112803
work_pressure                             27918
cgpa                                     112802
study_satisfaction                       112803
job_satisfaction                          27910
sleep_duration                                0
dietary_habits                                4
degree                                        2
have_you_ever_had_suicidal_thoughts_?         0
work/study_hours                              0
financial_stress                              4
family_history_of_mental_illness              0
depression                                    0
dtype: int64

### **Cleaning "Profession" Column**

In [91]:
df[df['profession'].isna()]

,name,gender,age,city,working_professional_or_student,profession,academic_pressure,work_pressure,cgpa,study_satisfaction,job_satisfaction,sleep_duration,dietary_habits,degree,have_you_ever_had_suicidal_thoughts_?,work/study_hours,financial_stress,family_history_of_mental_illness,depression
id,,,,,,,,,,,,,,,,,,,
2,Yuvraj,Male,33.0,Visakhapatnam,Student,NaN,5.0,NaN,8.97,2.0,NaN,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
8,Aishwarya,Female,24.0,Bangalore,Student,NaN,2.0,NaN,5.90,5.0,NaN,5-6 hours,Moderate,BSc,No,3.0,2.0,Yes,0
26,Aditya,Male,31.0,Srinagar,Student,NaN,3.0,NaN,7.03,5.0,NaN,Less than 5 hours,Healthy,BA,No,9.0,1.0,Yes,0
29,Kashish,Female,19.0,Agra,Working Professional,NaN,NaN,1.0,NaN,NaN,5.0,More than 8 hours,Healthy,Class 12,No,5.0,2.0,No,0
30,Prisha,Female,28.0,Varanasi,Student,NaN,3.0,NaN,5.59,2.0,NaN,7-8 hours,Moderate,BCA,Yes,4.0,5.0,Yes,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140686,Rohan,Male,27.0,Ludhiana,Student,NaN,2.0,NaN,9.40,3.0,NaN,Less than 5 hours,Healthy,MSc,No,0.0,3.0,Yes,0
140689,Ayaan,Male,31.0,Faridabad,Student,NaN,3.0,NaN,6.61,4.0,NaN,5-6 hours,Unhealthy,MD,No,12.0,2.0,No,0
140690,Rashi,Female,18.0,Ludhiana,Student,NaN,5.0,NaN,6.88,2.0,NaN,Less than 5 hours,Healthy,Class 12,Yes,10.0,5.0,No,1


In [92]:
df.loc[df['working_professional_or_student'] == "Student", 'profession'] = df.loc[df['working_professional_or_student'] == "Student", 'profession'].fillna('Student')


 **Cleaned "profeesion" to `Student` where "working_professional_or_student"  is `Student`  **

###  Cleaning "**Academic_pressure**" Column

In [93]:
df[df['academic_pressure'].isna()]


,name,gender,age,city,working_professional_or_student,profession,academic_pressure,work_pressure,cgpa,study_satisfaction,job_satisfaction,sleep_duration,dietary_habits,degree,have_you_ever_had_suicidal_thoughts_?,work/study_hours,financial_stress,family_history_of_mental_illness,depression
id,,,,,,,,,,,,,,,,,,,
0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,NaN,5.0,NaN,NaN,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0
1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,NaN,4.0,NaN,NaN,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1
3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,NaN,5.0,NaN,NaN,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1
4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,NaN,1.0,NaN,NaN,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0
5,Vani,Female,59.0,Ahmedabad,Working Professional,Finanancial Analyst,NaN,2.0,NaN,NaN,5.0,5-6 hours,Healthy,MCA,No,7.0,5.0,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140694,Ishaani,Female,45.0,Ahmedabad,Working Professional,Teacher,NaN,2.0,NaN,NaN,5.0,Less than 5 hours,Moderate,B.Ed,Yes,1.0,5.0,No,0
140695,Vidya,Female,18.0,Ahmedabad,Working Professional,NaN,NaN,5.0,NaN,NaN,4.0,5-6 hours,Unhealthy,Class 12,No,2.0,4.0,Yes,1
140696,Lata,Female,41.0,Hyderabad,Working Professional,Content Writer,NaN,5.0,NaN,NaN,4.0,7-8 hours,Moderate,B.Tech,Yes,6.0,5.0,Yes,0


**Setting "academic_pressure" to `0.0` Where "working_professional_or_student" is `Working Professional`**

In [94]:
df.loc[df['working_professional_or_student'] == "Working Professional", 'academic_pressure'] = df.loc[df['working_professional_or_student'] == "Working Professional", 'academic_pressure'].fillna(0.0)

**Setting "academic_pressure"  `NaN` to mean Value Where "working_professional_or_student" is `student`**

In [95]:
df.loc[df['working_professional_or_student'] == "Student", 'academic_pressure'] = df.loc[df['working_professional_or_student'] == "Student", 'academic_pressure'].fillna(df['academic_pressure'].mean())

### Cleaning "Work_pressure" Column

In [96]:
df[df['work_pressure'].isna()]

,name,gender,age,city,working_professional_or_student,profession,academic_pressure,work_pressure,cgpa,study_satisfaction,job_satisfaction,sleep_duration,dietary_habits,degree,have_you_ever_had_suicidal_thoughts_?,work/study_hours,financial_stress,family_history_of_mental_illness,depression
id,,,,,,,,,,,,,,,,,,,
2,Yuvraj,Male,33.0,Visakhapatnam,Student,Student,5.0,NaN,8.97,2.0,NaN,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
8,Aishwarya,Female,24.0,Bangalore,Student,Student,2.0,NaN,5.90,5.0,NaN,5-6 hours,Moderate,BSc,No,3.0,2.0,Yes,0
26,Aditya,Male,31.0,Srinagar,Student,Student,3.0,NaN,7.03,5.0,NaN,Less than 5 hours,Healthy,BA,No,9.0,1.0,Yes,0
30,Prisha,Female,28.0,Varanasi,Student,Student,3.0,NaN,5.59,2.0,NaN,7-8 hours,Moderate,BCA,Yes,4.0,5.0,Yes,1
32,Chhavi,Female,25.0,Jaipur,Student,Student,4.0,NaN,8.13,3.0,NaN,5-6 hours,Moderate,M.Tech,Yes,1.0,1.0,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140685,Aaradhya,Female,27.0,Surat,Student,Student,5.0,NaN,5.75,5.0,NaN,5-6 hours,Unhealthy,Class 12,Yes,7.0,1.0,Yes,0
140686,Rohan,Male,27.0,Ludhiana,Student,Student,2.0,NaN,9.40,3.0,NaN,Less than 5 hours,Healthy,MSc,No,0.0,3.0,Yes,0
140689,Ayaan,Male,31.0,Faridabad,Student,Student,3.0,NaN,6.61,4.0,NaN,5-6 hours,Unhealthy,MD,No,12.0,2.0,No,0


**Setting "Work_pressure" to `0.0` Where "working_professional_or_student" is `Student|`**

In [97]:
df.loc[df['working_professional_or_student'] == "Student", 'work_pressure'] = df.loc[df['working_professional_or_student'] == "Student", 'work_pressure'].fillna(0.0)

In [98]:
df[df['work_pressure'].isna()]

,name,gender,age,city,working_professional_or_student,profession,academic_pressure,work_pressure,cgpa,study_satisfaction,job_satisfaction,sleep_duration,dietary_habits,degree,have_you_ever_had_suicidal_thoughts_?,work/study_hours,financial_stress,family_history_of_mental_illness,depression
id,,,,,,,,,,,,,,,,,,,
15846,Shrey,Male,53.0,Vasai-Virar,Working Professional,NaN,0.0,NaN,NaN,NaN,5.0,More than 8 hours,Unhealthy,Class 12,No,12.0,1.0,No,0
18752,Aarti,Female,18.0,Kalyan,Working Professional,NaN,2.0,NaN,8.14,5.0,NaN,Less than 5 hours,Healthy,Class 12,Yes,7.0,1.0,Yes,1
34694,Kriti,Female,46.0,Meerut,Working Professional,NaN,0.0,NaN,NaN,NaN,NaN,5-6 hours,Moderate,Class 12,Yes,12.0,3.0,No,0
35612,Siddhesh,Male,27.0,Vadodara,Working Professional,NaN,0.0,NaN,NaN,NaN,NaN,Less than 5 hours,Unhealthy,B.Pharm,No,11.0,3.0,No,1
41915,Advait,Male,18.0,Kolkata,Working Professional,NaN,4.0,NaN,8.04,3.0,NaN,5-6 hours,Healthy,Class 12,Yes,1.0,5.0,Yes,1
55827,Prachi,Female,19.0,Kalyan,Working Professional,NaN,3.0,NaN,8.11,2.0,NaN,5-6 hours,Unhealthy,Class 12,Yes,4.0,4.0,No,1
57196,Manvi,Female,20.0,Visakhapatnam,Working Professional,NaN,0.0,NaN,NaN,NaN,NaN,Less than 5 hours,Moderate,Class 12,No,0.0,4.0,Yes,1
60348,Nishant,Male,29.0,Agra,Working Professional,NaN,0.0,NaN,9.72,2.0,NaN,5-6 hours,Moderate,B.Ed,No,7.0,2.0,Yes,0
76104,Vanya,Female,20.0,Nashik,Working Professional,NaN,0.0,NaN,NaN,NaN,1.0,7-8 hours,Moderate,Class 12,No,11.0,1.0,No,1


**Setting "academic_pressure" `NaN` to mean Value Where "working_professional_or_student" is `Student`**

In [99]:
df.loc[df['working_professional_or_student'] == "Working Professional", 'work_pressure'] = df.loc[df['working_professional_or_student'] == "Working Professional", 'work_pressure'].fillna(df['work_pressure'].mean())

In [100]:
df[df['study_satisfaction'].isna()]

,name,gender,age,city,working_professional_or_student,profession,academic_pressure,work_pressure,cgpa,study_satisfaction,job_satisfaction,sleep_duration,dietary_habits,degree,have_you_ever_had_suicidal_thoughts_?,work/study_hours,financial_stress,family_history_of_mental_illness,depression
id,,,,,,,,,,,,,,,,,,,
0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,0.0,5.0,NaN,NaN,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0
1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,0.0,4.0,NaN,NaN,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1
3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,0.0,5.0,NaN,NaN,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1
4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,0.0,1.0,NaN,NaN,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0
5,Vani,Female,59.0,Ahmedabad,Working Professional,Finanancial Analyst,0.0,2.0,NaN,NaN,5.0,5-6 hours,Healthy,MCA,No,7.0,5.0,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140694,Ishaani,Female,45.0,Ahmedabad,Working Professional,Teacher,0.0,2.0,NaN,NaN,5.0,Less than 5 hours,Moderate,B.Ed,Yes,1.0,5.0,No,0
140695,Vidya,Female,18.0,Ahmedabad,Working Professional,NaN,0.0,5.0,NaN,NaN,4.0,5-6 hours,Unhealthy,Class 12,No,2.0,4.0,Yes,1
140696,Lata,Female,41.0,Hyderabad,Working Professional,Content Writer,0.0,5.0,NaN,NaN,4.0,7-8 hours,Moderate,B.Tech,Yes,6.0,5.0,Yes,0


**Setting "Study_satisifcation" to `0.0` Where "working_professional_or_student" is `Working Professional`**

In [101]:
df.loc[df['working_professional_or_student'] == "Working Professional", 'study_satisfaction'] = df.loc[df['working_professional_or_student'] == "Working Professional", 'study_satisfaction'].fillna(0.0)
df.loc[df['working_professional_or_student'] == "Student", 'study_satisfaction'] = df.loc[df['working_professional_or_student'] == "Student", 'study_satisfaction'].fillna(df['study_satisfaction'].mean())

In [102]:
df['study_satisfaction'].isna().sum()

0

In [103]:
df[df['job_satisfaction'].isna()]

,name,gender,age,city,working_professional_or_student,profession,academic_pressure,work_pressure,cgpa,study_satisfaction,job_satisfaction,sleep_duration,dietary_habits,degree,have_you_ever_had_suicidal_thoughts_?,work/study_hours,financial_stress,family_history_of_mental_illness,depression
id,,,,,,,,,,,,,,,,,,,
2,Yuvraj,Male,33.0,Visakhapatnam,Student,Student,5.0,0.0,8.97,2.0,NaN,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
8,Aishwarya,Female,24.0,Bangalore,Student,Student,2.0,0.0,5.90,5.0,NaN,5-6 hours,Moderate,BSc,No,3.0,2.0,Yes,0
26,Aditya,Male,31.0,Srinagar,Student,Student,3.0,0.0,7.03,5.0,NaN,Less than 5 hours,Healthy,BA,No,9.0,1.0,Yes,0
30,Prisha,Female,28.0,Varanasi,Student,Student,3.0,0.0,5.59,2.0,NaN,7-8 hours,Moderate,BCA,Yes,4.0,5.0,Yes,1
32,Chhavi,Female,25.0,Jaipur,Student,Student,4.0,0.0,8.13,3.0,NaN,5-6 hours,Moderate,M.Tech,Yes,1.0,1.0,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140685,Aaradhya,Female,27.0,Surat,Student,Student,5.0,0.0,5.75,5.0,NaN,5-6 hours,Unhealthy,Class 12,Yes,7.0,1.0,Yes,0
140686,Rohan,Male,27.0,Ludhiana,Student,Student,2.0,0.0,9.40,3.0,NaN,Less than 5 hours,Healthy,MSc,No,0.0,3.0,Yes,0
140689,Ayaan,Male,31.0,Faridabad,Student,Student,3.0,0.0,6.61,4.0,NaN,5-6 hours,Unhealthy,MD,No,12.0,2.0,No,0


**Setting "job_satisfication" to `0.0` Where "working_professional_or_student" is `Working Professional`**

In [104]:
df.loc[df['working_professional_or_student'] == "Student", 'job_satisfaction'] = df.loc[df['working_professional_or_student'] == "Student", 'job_satisfaction'].fillna(0.0)
df.loc[df['working_professional_or_student'] == "Working Professional", 'job_satisfaction'] = df.loc[df['working_professional_or_student'] == "Working Professional", 'job_satisfaction'].fillna(df['job_satisfaction'].mean())

**Dropping Remaining `Null` Values**

In [105]:
df.dropna(subset=['profession', 'dietary_habits','degree','financial_stress'], inplace=True)

In [106]:
df.head()

,name,gender,age,city,working_professional_or_student,profession,academic_pressure,work_pressure,cgpa,study_satisfaction,job_satisfaction,sleep_duration,dietary_habits,degree,have_you_ever_had_suicidal_thoughts_?,work/study_hours,financial_stress,family_history_of_mental_illness,depression
id,,,,,,,,,,,,,,,,,,,
0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,0.0,5.0,NaN,0.0,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0
1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,0.0,4.0,NaN,0.0,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1
2,Yuvraj,Male,33.0,Visakhapatnam,Student,Student,5.0,0.0,8.97,2.0,0.0,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,0.0,5.0,NaN,0.0,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1
4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,0.0,1.0,NaN,0.0,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0


**Changing Dtype of "Age" to `Float` to `Int`**

In [107]:
df['age']=df['age'].astype('int')

**function for Value_counts**

In [108]:
def VC(col):
    for c in col:
        print(df[c].value_counts())
        print("\t  = = = = = = = = = = = = = = = = = = = \t")

In [109]:
VC(['city','working_professional_or_student','profession','dietary_habits','sleep_duration','degree','have_you_ever_had_suicidal_thoughts_?'])

city
Kalyan         6294
Patna          5663
Vasai-Virar    5387
Kolkata        5327
Meerut         5271
               ... 
Moreadhyay        1
Morena            1
Ishkarsh          1
Kashk             1
Malyansh          1
Name: count, Length: 97, dtype: int64
	  = = = = = = = = = = = = = = = = = = = 	
working_professional_or_student
Working Professional    104034
Student                  27896
Name: count, dtype: int64
	  = = = = = = = = = = = = = = = = = = = 	
profession
Student           27869
Teacher           24906
Content Writer     7814
Architect          4370
Consultant         4228
                  ...  
Moderate              1
Analyst               1
Pranav                1
Visakhapatnam         1
Yuvraj                1
Name: count, Length: 64, dtype: int64
	  = = = = = = = = = = = = = = = = = = = 	
dietary_habits
Moderate             46891
Unhealthy            42793
Healthy              42224
Yes                      2
No                       2
BSc                      

**Removing the Categories which has less than 10 Value_counts**

In [110]:
city_counts = df['city'].value_counts()
threshold = 10
cities_to_keep = city_counts[city_counts > threshold].index
df = df[df['city'].isin(cities_to_keep)]

In [111]:
degree_counts = df['degree'].value_counts()
threshold = 10
Degree_to_keep = degree_counts[degree_counts > threshold].index
df = df[df['degree'].isin(Degree_to_keep)]

In [112]:
profession_counts = df['profession'].value_counts()
threshold = 10
profession_to_keep = profession_counts[profession_counts > threshold].index
df = df[df['profession'].isin(profession_to_keep)]

In [113]:
# Cleaning `sleep_duration`
# Define a map to consolidate ranges
sleep_map = {
    'Less than 5 hours': '0-5 hours',
    'More than 8 hours': '8+ hours',
    '3-4 hours': '0-5 hours',
    '5-6 hours': '5-6 hours',
    '7-8 hours': '7-8 hours',
    # Add more mappings as needed
}
df['sleep_duration'] = df['sleep_duration'].replace(sleep_map)

valid_sleep = ['0-5 hours', '8+ hours', '5-6 hours','7-8 hours']
df['sleep_duration'] = df['sleep_duration'].where(df['sleep_duration'].isin(valid_sleep))

In [114]:
# Cleaning `dietary_habits`
# Drop unrelated or incorrect entries
valid_diet = ['Healthy', 'Unhealthy', 'Moderate']
df['dietary_habits'] = df['dietary_habits'].where(df['dietary_habits'].isin(valid_diet))

In [115]:
df.head()

,name,gender,age,city,working_professional_or_student,profession,academic_pressure,work_pressure,cgpa,study_satisfaction,job_satisfaction,sleep_duration,dietary_habits,degree,have_you_ever_had_suicidal_thoughts_?,work/study_hours,financial_stress,family_history_of_mental_illness,depression
id,,,,,,,,,,,,,,,,,,,
0,Aaradhya,Female,49,Ludhiana,Working Professional,Chef,0.0,5.0,NaN,0.0,2.0,8+ hours,Healthy,BHM,No,1.0,2.0,No,0
1,Vivan,Male,26,Varanasi,Working Professional,Teacher,0.0,4.0,NaN,0.0,3.0,0-5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1
2,Yuvraj,Male,33,Visakhapatnam,Student,Student,5.0,0.0,8.97,2.0,0.0,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
3,Yuvraj,Male,22,Mumbai,Working Professional,Teacher,0.0,5.0,NaN,0.0,1.0,0-5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1
4,Rhea,Female,30,Kanpur,Working Professional,Business Analyst,0.0,1.0,NaN,0.0,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0


### Now Evrything is Clean And Good For Modeling

## Label Encoding

In [116]:
col = ['degree','profession','city','dietary_habits','gender','working_professional_or_student','sleep_duration','have_you_ever_had_suicidal_thoughts_?','family_history_of_mental_illness']


def label_encode(df, column_name):
    le = LabelEncoder()
    for col in column_name:
        df[col] = le.fit_transform(df[col])
    return df

In [117]:
label_encode(df,col)

,name,gender,age,city,working_professional_or_student,profession,academic_pressure,work_pressure,cgpa,study_satisfaction,job_satisfaction,sleep_duration,dietary_habits,degree,have_you_ever_had_suicidal_thoughts_?,work/study_hours,financial_stress,family_history_of_mental_illness,depression
id,,,,,,,,,,,,,,,,,,,
0,Aaradhya,0,49,15,1,3,0.0,5.0,NaN,0.0,2.0,3,0,9,0,1.0,2.0,0,0
1,Vivan,1,26,27,1,33,0.0,4.0,NaN,0.0,3.0,0,2,12,1,7.0,3.0,0,1
2,Yuvraj,1,33,29,0,32,5.0,0.0,8.97,2.0,0.0,1,0,3,1,3.0,1.0,0,1
3,Yuvraj,1,22,17,1,33,0.0,5.0,NaN,0.0,1.0,0,1,6,1,10.0,1.0,1,1
4,Rhea,0,30,12,1,2,0.0,1.0,NaN,0.0,1.0,1,2,6,1,9.0,4.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140694,Ishaani,0,45,1,1,33,0.0,2.0,NaN,0.0,5.0,0,1,2,1,1.0,5.0,0,0
140696,Lata,0,41,8,1,7,0.0,5.0,NaN,0.0,4.0,2,1,4,1,6.0,5.0,1,0
140697,Aanchal,0,24,13,1,23,0.0,3.0,NaN,0.0,1.0,3,1,1,0,4.0,4.0,0,0


In [118]:
df_copy = df.copy()

# Test Data

In [119]:
test=pd.read_csv('/kaggle/input/playground-series-s4e11/test.csv',index_col=['id'])
test.head()

,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness
id,,,,,,,,,,,,,,,,,,
140700,Shivam,Male,53.0,Visakhapatnam,Working Professional,Judge,NaN,2.0,NaN,NaN,5.0,Less than 5 hours,Moderate,LLB,No,9.0,3.0,Yes
140701,Sanya,Female,58.0,Kolkata,Working Professional,Educational Consultant,NaN,2.0,NaN,NaN,4.0,Less than 5 hours,Moderate,B.Ed,No,6.0,4.0,No
140702,Yash,Male,53.0,Jaipur,Working Professional,Teacher,NaN,4.0,NaN,NaN,1.0,7-8 hours,Moderate,B.Arch,Yes,12.0,4.0,No
140703,Nalini,Female,23.0,Rajkot,Student,NaN,5.0,NaN,6.84,1.0,NaN,More than 8 hours,Moderate,BSc,Yes,10.0,4.0,No
140704,Shaurya,Male,47.0,Kalyan,Working Professional,Teacher,NaN,5.0,NaN,NaN,5.0,7-8 hours,Moderate,BCA,Yes,3.0,4.0,No


### **Did Same processing as for Train Data**

In [120]:
# Standardize column names
test.columns = test.columns.str.strip().str.replace(" ", "_").str.lower()

In [121]:
test.loc[test['working_professional_or_student'] == "Student", 'profession'] = test.loc[test['working_professional_or_student'] == "Student", 'profession'].fillna('Student')


In [122]:
test[test['academic_pressure'].isna()]

,name,gender,age,city,working_professional_or_student,profession,academic_pressure,work_pressure,cgpa,study_satisfaction,job_satisfaction,sleep_duration,dietary_habits,degree,have_you_ever_had_suicidal_thoughts_?,work/study_hours,financial_stress,family_history_of_mental_illness
id,,,,,,,,,,,,,,,,,,
140700,Shivam,Male,53.0,Visakhapatnam,Working Professional,Judge,NaN,2.0,NaN,NaN,5.0,Less than 5 hours,Moderate,LLB,No,9.0,3.0,Yes
140701,Sanya,Female,58.0,Kolkata,Working Professional,Educational Consultant,NaN,2.0,NaN,NaN,4.0,Less than 5 hours,Moderate,B.Ed,No,6.0,4.0,No
140702,Yash,Male,53.0,Jaipur,Working Professional,Teacher,NaN,4.0,NaN,NaN,1.0,7-8 hours,Moderate,B.Arch,Yes,12.0,4.0,No
140704,Shaurya,Male,47.0,Kalyan,Working Professional,Teacher,NaN,5.0,NaN,NaN,5.0,7-8 hours,Moderate,BCA,Yes,3.0,4.0,No
140705,Kartik,Male,29.0,Mumbai,Working Professional,Customer Support,NaN,2.0,NaN,NaN,3.0,More than 8 hours,Moderate,B.Com,No,3.0,2.0,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234492,Advait,Male,23.0,Delhi,Working Professional,Consultant,NaN,2.0,NaN,NaN,3.0,Less than 5 hours,Moderate,B.Com,Yes,12.0,4.0,No
234495,Zoya,Female,49.0,Jaipur,Working Professional,Pilot,NaN,3.0,NaN,NaN,5.0,Less than 5 hours,Moderate,BSc,Yes,2.0,2.0,Yes
234496,Shlok,Male,29.0,Ahmedabad,Working Professional,Pilot,NaN,5.0,NaN,NaN,1.0,7-8 hours,Moderate,BE,Yes,11.0,3.0,Yes


In [123]:
test.loc[test['working_professional_or_student'] == "Working Professional", 'academic_pressure'] = test.loc[test['working_professional_or_student'] == "Working Professional", 'academic_pressure'].fillna(0.0)

In [124]:
test.loc[test['working_professional_or_student'] == "Student", 'academic_pressure'] = test.loc[test['working_professional_or_student'] == "Student", 'academic_pressure'].fillna(test['academic_pressure'].mean())

In [125]:
test.loc[test['working_professional_or_student'] == "Student", 'academic_pressure'].isna().sum()

0

In [126]:
test[test['work_pressure'].isna()]

,name,gender,age,city,working_professional_or_student,profession,academic_pressure,work_pressure,cgpa,study_satisfaction,job_satisfaction,sleep_duration,dietary_habits,degree,have_you_ever_had_suicidal_thoughts_?,work/study_hours,financial_stress,family_history_of_mental_illness
id,,,,,,,,,,,,,,,,,,
140703,Nalini,Female,23.0,Rajkot,Student,Student,5.0,NaN,6.84,1.0,NaN,More than 8 hours,Moderate,BSc,Yes,10.0,4.0,No
140708,Navya,Female,21.0,Surat,Student,Student,1.0,NaN,7.39,3.0,NaN,Less than 5 hours,Healthy,BBA,No,8.0,1.0,Yes
140719,Kiran,Male,28.0,Lucknow,Student,Student,5.0,NaN,6.03,3.0,NaN,7-8 hours,Moderate,MA,No,11.0,3.0,Yes
140720,Rashi,Female,26.0,Ghaziabad,Student,Student,3.0,NaN,8.63,5.0,NaN,More than 8 hours,Moderate,BBA,Yes,10.0,4.0,Yes
140721,Manan,Male,32.0,Visakhapatnam,Student,Student,3.0,NaN,8.61,1.0,NaN,Less than 5 hours,Moderate,M.Ed,Yes,11.0,5.0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234485,Ishaan,Male,33.0,Agra,Student,Student,5.0,NaN,5.74,2.0,NaN,7-8 hours,Moderate,B.Com,Yes,12.0,5.0,Yes
234490,Abhishek,Male,19.0,Nashik,Student,Student,4.0,NaN,8.96,3.0,NaN,7-8 hours,Unhealthy,Class 12,No,2.0,2.0,Yes
234493,Aohi,Female,29.0,Vadodara,Student,Student,2.0,NaN,5.58,5.0,NaN,More than 8 hours,Healthy,Class 12,Yes,8.0,2.0,Yes


In [127]:
test.loc[test['working_professional_or_student'] == "Student", 'work_pressure'] = test.loc[test['working_professional_or_student'] == "Student", 'work_pressure'].fillna(0.0)

In [128]:
test[test['work_pressure'].isna()]

,name,gender,age,city,working_professional_or_student,profession,academic_pressure,work_pressure,cgpa,study_satisfaction,job_satisfaction,sleep_duration,dietary_habits,degree,have_you_ever_had_suicidal_thoughts_?,work/study_hours,financial_stress,family_history_of_mental_illness
id,,,,,,,,,,,,,,,,,,
145631,Tina,Female,18.0,Srinagar,Working Professional,NaN,2.0,NaN,8.04,5.0,NaN,More than 8 hours,Unhealthy,Class 12,Yes,6.0,4.0,No
149705,Anand,Male,18.0,Vasai-Virar,Working Professional,NaN,0.0,NaN,NaN,NaN,2.0,Less than 5 hours,Unhealthy,Class 12,Yes,9.0,4.0,No
160926,Himani,Female,19.0,Rajkot,Working Professional,NaN,0.0,NaN,NaN,NaN,2.0,Less than 5 hours,Moderate,Class 12,Yes,0.0,5.0,Yes
171833,Sanya,Female,33.0,Kolkata,Working Professional,NaN,0.0,NaN,9.88,5.0,NaN,More than 8 hours,Healthy,MCA,Yes,11.0,1.0,Yes
172498,Janvi,Female,28.0,Mumbai,Working Professional,NaN,0.0,NaN,NaN,NaN,NaN,5-6 hours,Unhealthy,BCA,Yes,8.0,4.0,No
174747,Saanvi,Female,50.0,Thane,Working Professional,NaN,0.0,NaN,NaN,NaN,NaN,Less than 5 hours,Unhealthy,Class 12,Yes,2.0,5.0,No
181776,Ila,Female,52.0,Kanpur,Working Professional,NaN,0.0,NaN,NaN,NaN,2.0,7-8 hours,Unhealthy,M.Pharm,Yes,3.0,3.0,No
189676,Tanisha,Female,37.0,Thane,Working Professional,NaN,0.0,NaN,NaN,NaN,NaN,Less than 5 hours,Moderate,LLB,Yes,0.0,2.0,Yes
190034,Vani,Female,19.0,Pune,Working Professional,NaN,2.0,NaN,NaN,4.0,NaN,Less than 5 hours,Moderate,Class 12,Yes,2.0,2.0,No


In [129]:
test.loc[test['working_professional_or_student'] == "Working Professional", 'work_pressure'] = test.loc[test['working_professional_or_student'] == "Working Professional", 'work_pressure'].fillna(test['work_pressure'].mean())

In [130]:
test[test['study_satisfaction'].isna()]

,name,gender,age,city,working_professional_or_student,profession,academic_pressure,work_pressure,cgpa,study_satisfaction,job_satisfaction,sleep_duration,dietary_habits,degree,have_you_ever_had_suicidal_thoughts_?,work/study_hours,financial_stress,family_history_of_mental_illness
id,,,,,,,,,,,,,,,,,,
140700,Shivam,Male,53.0,Visakhapatnam,Working Professional,Judge,0.0,2.0,NaN,NaN,5.0,Less than 5 hours,Moderate,LLB,No,9.0,3.0,Yes
140701,Sanya,Female,58.0,Kolkata,Working Professional,Educational Consultant,0.0,2.0,NaN,NaN,4.0,Less than 5 hours,Moderate,B.Ed,No,6.0,4.0,No
140702,Yash,Male,53.0,Jaipur,Working Professional,Teacher,0.0,4.0,NaN,NaN,1.0,7-8 hours,Moderate,B.Arch,Yes,12.0,4.0,No
140704,Shaurya,Male,47.0,Kalyan,Working Professional,Teacher,0.0,5.0,NaN,NaN,5.0,7-8 hours,Moderate,BCA,Yes,3.0,4.0,No
140705,Kartik,Male,29.0,Mumbai,Working Professional,Customer Support,0.0,2.0,NaN,NaN,3.0,More than 8 hours,Moderate,B.Com,No,3.0,2.0,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234492,Advait,Male,23.0,Delhi,Working Professional,Consultant,0.0,2.0,NaN,NaN,3.0,Less than 5 hours,Moderate,B.Com,Yes,12.0,4.0,No
234495,Zoya,Female,49.0,Jaipur,Working Professional,Pilot,0.0,3.0,NaN,NaN,5.0,Less than 5 hours,Moderate,BSc,Yes,2.0,2.0,Yes
234496,Shlok,Male,29.0,Ahmedabad,Working Professional,Pilot,0.0,5.0,NaN,NaN,1.0,7-8 hours,Moderate,BE,Yes,11.0,3.0,Yes


In [131]:
test.loc[test['working_professional_or_student'] == "Working Professional", 'study_satisfaction'] = test.loc[test['working_professional_or_student'] == "Working Professional", 'study_satisfaction'].fillna(0.0)
test.loc[test['working_professional_or_student'] == "Student", 'study_satisfaction'] = test.loc[test['working_professional_or_student'] == "Student", 'study_satisfaction'].fillna(test['study_satisfaction'].mean())

In [132]:
test['study_satisfaction'].isna().sum()

0

In [133]:
test[test['job_satisfaction'].isna()]

,name,gender,age,city,working_professional_or_student,profession,academic_pressure,work_pressure,cgpa,study_satisfaction,job_satisfaction,sleep_duration,dietary_habits,degree,have_you_ever_had_suicidal_thoughts_?,work/study_hours,financial_stress,family_history_of_mental_illness
id,,,,,,,,,,,,,,,,,,
140703,Nalini,Female,23.0,Rajkot,Student,Student,5.0,0.0,6.84,1.0,NaN,More than 8 hours,Moderate,BSc,Yes,10.0,4.0,No
140708,Navya,Female,21.0,Surat,Student,Student,1.0,0.0,7.39,3.0,NaN,Less than 5 hours,Healthy,BBA,No,8.0,1.0,Yes
140719,Kiran,Male,28.0,Lucknow,Student,Student,5.0,0.0,6.03,3.0,NaN,7-8 hours,Moderate,MA,No,11.0,3.0,Yes
140720,Rashi,Female,26.0,Ghaziabad,Student,Student,3.0,0.0,8.63,5.0,NaN,More than 8 hours,Moderate,BBA,Yes,10.0,4.0,Yes
140721,Manan,Male,32.0,Visakhapatnam,Student,Student,3.0,0.0,8.61,1.0,NaN,Less than 5 hours,Moderate,M.Ed,Yes,11.0,5.0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234485,Ishaan,Male,33.0,Agra,Student,Student,5.0,0.0,5.74,2.0,NaN,7-8 hours,Moderate,B.Com,Yes,12.0,5.0,Yes
234490,Abhishek,Male,19.0,Nashik,Student,Student,4.0,0.0,8.96,3.0,NaN,7-8 hours,Unhealthy,Class 12,No,2.0,2.0,Yes
234493,Aohi,Female,29.0,Vadodara,Student,Student,2.0,0.0,5.58,5.0,NaN,More than 8 hours,Healthy,Class 12,Yes,8.0,2.0,Yes


In [134]:
test.loc[test['working_professional_or_student'] == "Student", 'job_satisfaction'] = test.loc[test['working_professional_or_student'] == "Student", 'job_satisfaction'].fillna(0.0)
test.loc[test['working_professional_or_student'] == "Working Professional", 'job_satisfaction'] = test.loc[test['working_professional_or_student'] == "Working Professional", 'job_satisfaction'].fillna(test['job_satisfaction'].mean())

In [135]:
test.dropna(subset=['profession', 'dietary_habits','degree','financial_stress'], inplace=True)

In [136]:
test['age']=test['age'].astype('int')

In [137]:
def VC(col):
    for c in col:
        print(test[c].value_counts())
        print("\t  = = = = = = = = = = = = = = = = = = = \t")

In [138]:
VC(['city','working_professional_or_student','profession','dietary_habits','sleep_duration','degree','have_you_ever_had_suicidal_thoughts_?'])

city
Kalyan         4188
Patna          3725
Vasai-Virar    3633
Kolkata        3504
Meerut         3444
               ... 
Keshav            1
Abhinav           1
Rolkata           1
Ghopal            1
No.12             1
Name: count, Length: 68, dtype: int64
	  = = = = = = = = = = = = = = = = = = = 	
working_professional_or_student
Working Professional    69139
Student                 18768
Name: count, dtype: int64
	  = = = = = = = = = = = = = = = = = = = 	
profession
Student            18751
Teacher            16384
Content Writer      5187
Architect           2982
Consultant          2920
                   ...  
Surgeon                1
No                     1
Unveil                 1
City Consultant        1
M.Pharm                1
Name: count, Length: 64, dtype: int64
	  = = = = = = = = = = = = = = = = = = = 	
dietary_habits
Moderate        31179
Unhealthy       28494
Healthy         28212
No                  5
More Healthy        2
Prachi              1
Male               

In [139]:
city_counts = test['city'].value_counts()
threshold = 10
cities_to_keep = city_counts[city_counts > threshold].index
test = test[test['city'].isin(cities_to_keep)]

In [140]:
degree_counts = test['degree'].value_counts()
threshold = 10
Degree_to_keep = degree_counts[degree_counts > threshold].index
test = test[test['degree'].isin(Degree_to_keep)]

In [141]:
profession_counts = test['profession'].value_counts()
threshold = 10
profession_to_keep = profession_counts[profession_counts > threshold].index
test = test[test['profession'].isin(profession_to_keep)]

In [142]:
# Cleaning `sleep_duration`
# Define a map to consolidate ranges
sleep_map = {
    'Less than 5 hours': '0-5 hours',
    'More than 8 hours': '8+ hours',
    '3-4 hours': '0-5 hours',
    '5-6 hours': '5-6 hours',
    '7-8 hours': '7-8 hours',
    # Add more mappings as needed
}
test['sleep_duration'] = test['sleep_duration'].replace(sleep_map)

valid_sleep = ['0-5 hours', '8+ hours', '5-6 hours','7-8 hours']
test['sleep_duration'] = test['sleep_duration'].where(test['sleep_duration'].isin(valid_sleep))

In [143]:
# Cleaning `dietary_habits`
# Drop unrelated or incorrect entries
valid_diet = ['Healthy', 'Unhealthy', 'Moderate']
test['dietary_habits'] = test['dietary_habits'].where(test['dietary_habits'].isin(valid_diet))

In [144]:
test.head()

,name,gender,age,city,working_professional_or_student,profession,academic_pressure,work_pressure,cgpa,study_satisfaction,job_satisfaction,sleep_duration,dietary_habits,degree,have_you_ever_had_suicidal_thoughts_?,work/study_hours,financial_stress,family_history_of_mental_illness
id,,,,,,,,,,,,,,,,,,
140700,Shivam,Male,53,Visakhapatnam,Working Professional,Judge,0.0,2.0,NaN,0.0,5.0,0-5 hours,Moderate,LLB,No,9.0,3.0,Yes
140701,Sanya,Female,58,Kolkata,Working Professional,Educational Consultant,0.0,2.0,NaN,0.0,4.0,0-5 hours,Moderate,B.Ed,No,6.0,4.0,No
140702,Yash,Male,53,Jaipur,Working Professional,Teacher,0.0,4.0,NaN,0.0,1.0,7-8 hours,Moderate,B.Arch,Yes,12.0,4.0,No
140703,Nalini,Female,23,Rajkot,Student,Student,5.0,0.0,6.84,1.0,0.0,8+ hours,Moderate,BSc,Yes,10.0,4.0,No
140704,Shaurya,Male,47,Kalyan,Working Professional,Teacher,0.0,5.0,NaN,0.0,5.0,7-8 hours,Moderate,BCA,Yes,3.0,4.0,No


In [145]:
test['sleep_duration'].dropna(inplace = True)

In [146]:
test[test['sleep_duration'].isna()]

,name,gender,age,city,working_professional_or_student,profession,academic_pressure,work_pressure,cgpa,study_satisfaction,job_satisfaction,sleep_duration,dietary_habits,degree,have_you_ever_had_suicidal_thoughts_?,work/study_hours,financial_stress,family_history_of_mental_illness
id,,,,,,,,,,,,,,,,,,
141223,Vani,Female,38,Varanasi,Working Professional,Lawyer,0.0,1.0,NaN,0.0,4.0,NaN,Unhealthy,LLM,No,1.0,4.0,No
144293,Rajat,Male,45,Delhi,Working Professional,Electrician,0.0,4.0,NaN,0.0,4.0,NaN,Unhealthy,PhD,No,0.0,1.0,No
145313,Rupal,Female,46,Varanasi,Working Professional,Consultant,0.0,1.0,NaN,0.0,4.0,NaN,Moderate,BE,Yes,5.0,5.0,Yes
149601,Rhea,Female,54,Patna,Working Professional,Content Writer,0.0,3.0,NaN,0.0,3.0,NaN,Moderate,M.Ed,No,0.0,5.0,Yes
153512,Anvi,Female,33,Vasai-Virar,Student,Student,3.0,0.0,9.97,1.0,0.0,NaN,Unhealthy,BSc,Yes,0.0,2.0,Yes
156298,Ansh,Male,38,Vasai-Virar,Working Professional,Content Writer,0.0,4.0,NaN,0.0,1.0,NaN,Unhealthy,BSc,Yes,6.0,4.0,Yes
157107,Aanchal,Female,27,Srinagar,Working Professional,Graphic Designer,0.0,4.0,NaN,0.0,1.0,NaN,Healthy,BSc,Yes,6.0,5.0,Yes
159769,Eshita,Female,20,Surat,Student,Student,1.0,0.0,8.63,5.0,0.0,NaN,Healthy,Class 12,Yes,7.0,2.0,Yes
161326,Aaradhya,Female,28,Bhopal,Working Professional,Teacher,0.0,5.0,NaN,0.0,1.0,NaN,Unhealthy,MBBS,Yes,11.0,4.0,Yes


In [147]:
test1 =test.copy()

In [148]:
col = ['dietary_habits','gender','working_professional_or_student','profession','degree','city','sleep_duration','have_you_ever_had_suicidal_thoughts_?','family_history_of_mental_illness']


def label_encode(test, column_name):
    le = LabelEncoder()
    for col in column_name:
        test[col] = le.fit_transform(test[col])
    return test

In [149]:
label_encode(test,col)

,name,gender,age,city,working_professional_or_student,profession,academic_pressure,work_pressure,cgpa,study_satisfaction,job_satisfaction,sleep_duration,dietary_habits,degree,have_you_ever_had_suicidal_thoughts_?,work/study_hours,financial_stress,family_history_of_mental_illness
id,,,,,,,,,,,,,,,,,,
140700,Shivam,1,53,29,1,20,0.0,2.0,NaN,0.0,5.0,0,1,12,0,9.0,3.0,1
140701,Sanya,0,58,13,1,12,0.0,2.0,NaN,0.0,4.0,0,1,2,0,6.0,4.0,0
140702,Yash,1,53,10,1,33,0.0,4.0,NaN,0.0,1.0,2,1,0,1,12.0,4.0,0
140703,Nalini,0,23,22,0,32,5.0,0.0,6.84,1.0,0.0,3,1,10,1,10.0,4.0,0
140704,Shaurya,1,47,11,1,33,0.0,5.0,NaN,0.0,5.0,2,1,7,1,3.0,4.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234495,Zoya,0,49,10,1,26,0.0,3.0,NaN,0.0,5.0,0,1,10,1,2.0,2.0,1
234496,Shlok,1,29,1,1,26,0.0,5.0,NaN,0.0,1.0,2,1,8,1,11.0,3.0,1
234497,Rishi,1,24,29,0,32,1.0,0.0,7.51,4.0,0.0,2,1,4,0,7.0,1.0,0


In [150]:
#!pip install pandas==1.4.0

In [151]:
# def one_hot_encode_multiple_columns(df, columns):
#     # Replace 'sparse' with 'sparse_output'
#     ohe = OneHotEncoder(sparse_output=False, drop='first')
#     # Fit and transform the specified columns
#     encoded = ohe.fit_transform(df[columns])
#     # Create a DataFrame with the encoded columns
#     encoded_df = pd.DataFrame(encoded, columns=ohe.get_feature_names_out(columns))
#     # Concatenate with the original DataFrame and drop the original columns
#     df = pd.concat([df.reset_index(drop=True), encoded_df], axis=1).drop(columns, axis=1)
#     return df

In [152]:
# For one-hot encoding
# df = one_hot_encode_multiple_columns(df, ['city', 'profession','degree'])

# Scaling the data

In [153]:
columns_to_scale = df_copy.columns.difference(['name','depression'])
scaler = StandardScaler()
# Fit and transform the specified columns
df_scaled = scaler.fit_transform(df_copy[columns_to_scale])
df_copy[columns_to_scale] = pd.DataFrame(df_scaled, columns=columns_to_scale, index=df_copy.index)
df_copy.head()

,name,gender,age,city,working_professional_or_student,profession,academic_pressure,work_pressure,cgpa,study_satisfaction,job_satisfaction,sleep_duration,dietary_habits,degree,have_you_ever_had_suicidal_thoughts_?,work/study_hours,financial_stress,family_history_of_mental_illness,depression
id,,,,,,,,,,,,,,,,,,,
0,Aaradhya,-1.105766,0.661355,0.000241,0.517717,-1.586476,-0.463995,1.531596,NaN,-0.459285,-0.197686,1.368983,-1.250727,-0.258354,-0.978341,-1.359133,-0.693750,-0.995189,0
1,Vivan,0.904350,-1.260907,1.399760,0.517717,0.973695,-0.463995,0.955571,NaN,-0.459285,0.375582,-1.300393,1.239382,0.114893,1.022138,0.196737,0.014412,-0.995189,1
2,Yuvraj,0.904350,-0.675871,1.633013,-1.931557,0.888356,3.028953,-1.348527,0.895475,1.016404,-1.344223,-0.410601,-1.250727,-1.004850,1.022138,-0.840510,-1.401912,-0.995189,1
3,Yuvraj,0.904350,-1.595213,0.233494,0.517717,0.973695,-0.463995,1.531596,NaN,-0.459285,-0.770955,-1.300393,-0.005673,-0.631602,1.022138,0.974672,-1.401912,1.004834,1
4,Rhea,-1.105766,-0.926600,-0.349639,0.517717,-1.671815,-0.463995,-0.772502,NaN,-0.459285,-0.770955,-0.410601,1.239382,-0.631602,1.022138,0.715360,0.722575,1.004834,0


# Training And Spliting

In [154]:
df['depression'].value_counts()

depression
0    109454
1     22231
Name: count, dtype: int64

In [155]:
X = df_copy.drop(columns=['name','cgpa','depression'])  # Features
y = df_copy['depression']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2022)


In [156]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((105348, 16), (26337, 16), (105348,), (26337,))

# Modeling

In [157]:
columns_to_scale = test.columns.difference(['name','cgpa'])

# Fit and transform the specified columns
test_scaled = scaler.fit_transform(test[columns_to_scale])
test[columns_to_scale] = pd.DataFrame(test_scaled, columns=columns_to_scale, index=test.index)
test.head()

,name,gender,age,city,working_professional_or_student,profession,academic_pressure,work_pressure,cgpa,study_satisfaction,job_satisfaction,sleep_duration,dietary_habits,degree,have_you_ever_had_suicidal_thoughts_?,work/study_hours,financial_stress,family_history_of_mental_illness
id,,,,,,,,,,,,,,,,,,
140700,Shivam,0.912373,1.000144,1.630198,0.521033,-0.132160,-0.466898,-0.198480,NaN,-0.460811,1.537056,-1.307128,-0.004566,0.124458,-0.981122,0.716088,0.023250,1.002739
140701,Sanya,-1.096043,1.417446,-0.229877,0.521033,-0.814741,-0.466898,-0.198480,NaN,-0.460811,0.961831,-1.307128,-0.004566,-1.121530,-0.981122,-0.061639,0.730857,-0.997268
140702,Yash,0.912373,1.000144,-0.578641,0.521033,0.977033,-0.466898,0.950230,NaN,-0.460811,-0.763844,0.470378,-0.004566,-1.370728,1.019241,1.493814,0.730857,-0.997268
140703,Nalini,-1.096043,-1.503669,0.816415,-1.919263,0.891710,2.995372,-1.347189,6.84,0.273808,-1.339068,1.359131,-0.004566,-0.124739,1.019241,0.975330,0.730857,-0.997268
140704,Shaurya,0.912373,0.499381,-0.462386,0.521033,0.977033,-0.466898,1.524584,NaN,-0.460811,1.537056,0.470378,-0.004566,-0.498536,1.019241,-0.839365,0.730857,-0.997268


## Logistic Regression

In [158]:
# Step 1: Train Logistic Regression model
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

# Step 2: Predictions
y_pred = log_reg.predict(X_test)

# Step 3: Model Evaluation
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Classification Report:\n{class_report}")


Accuracy: 0.9415
Confusion Matrix:
[[21295   690]
 [  852  3500]]
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.97      0.97     21985
           1       0.84      0.80      0.82      4352

    accuracy                           0.94     26337
   macro avg       0.90      0.89      0.89     26337
weighted avg       0.94      0.94      0.94     26337



test.head(

### Testing on Unseen Data "Test"

In [159]:
test.drop(columns=['name','cgpa'],inplace=True)

y_pred = log_reg.predict(test)

test1['predicted_depression'] = y_pred
test1.head()

,name,gender,age,city,working_professional_or_student,profession,academic_pressure,work_pressure,cgpa,study_satisfaction,job_satisfaction,sleep_duration,dietary_habits,degree,have_you_ever_had_suicidal_thoughts_?,work/study_hours,financial_stress,family_history_of_mental_illness,predicted_depression
id,,,,,,,,,,,,,,,,,,,
140700,Shivam,Male,53,Visakhapatnam,Working Professional,Judge,0.0,2.0,NaN,0.0,5.0,0-5 hours,Moderate,LLB,No,9.0,3.0,Yes,0
140701,Sanya,Female,58,Kolkata,Working Professional,Educational Consultant,0.0,2.0,NaN,0.0,4.0,0-5 hours,Moderate,B.Ed,No,6.0,4.0,No,0
140702,Yash,Male,53,Jaipur,Working Professional,Teacher,0.0,4.0,NaN,0.0,1.0,7-8 hours,Moderate,B.Arch,Yes,12.0,4.0,No,0
140703,Nalini,Female,23,Rajkot,Student,Student,5.0,0.0,6.84,1.0,0.0,8+ hours,Moderate,BSc,Yes,10.0,4.0,No,1
140704,Shaurya,Male,47,Kalyan,Working Professional,Teacher,0.0,5.0,NaN,0.0,5.0,7-8 hours,Moderate,BCA,Yes,3.0,4.0,No,0


## Decision Tree

In [160]:
# Decision Tree Model
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

# Predictions and Evaluation
y_pred_dt = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test,y_pred_dt))
conf_matrix = confusion_matrix(y_test, y_pred_dt)
class_report = classification_report(y_test, y_pred_dt)

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Classification Report:\n{class_report}")


Accuracy: 0.9084557846375821
Accuracy: 0.9415
Confusion Matrix:
[[20680  1305]
 [ 1106  3246]]
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.94      0.94     21985
           1       0.71      0.75      0.73      4352

    accuracy                           0.91     26337
   macro avg       0.83      0.84      0.84     26337
weighted avg       0.91      0.91      0.91     26337



## Random Forest

In [161]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Random Forest Model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Predictions and Evaluation
y_pred_Rf = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred_Rf))
conf_matrix = confusion_matrix(y_test, y_pred_Rf)
class_report = classification_report(y_test, y_pred_Rf)

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Classification Report:\n{class_report}")


Accuracy: 0.9411474351672552
Accuracy: 0.9415
Confusion Matrix:
[[21326   659]
 [  891  3461]]
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.97      0.96     21985
           1       0.84      0.80      0.82      4352

    accuracy                           0.94     26337
   macro avg       0.90      0.88      0.89     26337
weighted avg       0.94      0.94      0.94     26337



## Voting Classifier

In [162]:
LR = LogisticRegression()
Tree = DecisionTreeClassifier()
RF = RandomForestClassifier()
SVM = SVC()
KNN = KNeighborsClassifier()
GB = GradientBoostingClassifier()
AB = AdaBoostClassifier()
LGBM = LGBMClassifier()

# Voting Classifier
model = VotingClassifier(
    estimators=[ ("Decision Tree", Tree),("Logistic Regression", LR) ,("Random Forest", RF),
               ("Support Vector Machine",SVM), ("k-nearest neighbor",KNN), ("Gradient Boosting",GB),
               ("AdaBoost",AB), ("LGBM", LGBM)],
    voting='hard',
)


# Preprocessing of training data, fit model
model.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
prediction = model.predict(X_test)


[LightGBM] [Info] Number of positive: 17879, number of negative: 87469
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007989 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 213
[LightGBM] [Info] Number of data points in the train set: 105348, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.169714 -> initscore=-1.587658
[LightGBM] [Info] Start training from score -1.587658


In [163]:
cm = confusion_matrix(prediction,y_test)
cm

array([[21347,   876],
       [  638,  3476]])

In [164]:
sk_report = classification_report(
    digits=2,
    y_true=prediction,
    y_pred=y_test)
print(sk_report)

              precision    recall  f1-score   support

           0       0.97      0.96      0.97     22223
           1       0.80      0.84      0.82      4114

    accuracy                           0.94     26337
   macro avg       0.88      0.90      0.89     26337
weighted avg       0.94      0.94      0.94     26337



In [165]:
# Predict on test data
test_pred = model.predict(test)

# Create submission DataFrame
submission = pd.DataFrame({
    'id': test.index,
    'Depression': test_pred
})

# Save submission to CSV
submission.to_csv('submission.csv', index=False)

-----


